In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date
from collections import Counter
from multiprocessing import Process, Queue
import pandas as pd
import time
import csv

class instagram_Data_Crawler:
    def instagram_Follows_Crawling(self, path = '/Users/markmac/Downloads/chromedriver'):
        """
            인스타그램 가계정안에 팔로우 계정ID 수집
            
            path: 크롬드라이버 설치 위치
        """
        self.path = path
        
        account_Id = [] # 인스타그램 계정ID 저장할 변수 선언
        
        driver = webdriver.Chrome(path) # 크롬브라우저 실행
        driver.get('https://www.instagram.com') # 인스타그램 URL 주소 입력
        
        time.sleep(1) # 1초 일시정지
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.izU2O > a'))).click() # 명시적으로 해당 태그가 나올때까지 기다린 후 로그인 버튼 클릭
        
        time.sleep(1)
        #driver.find_element_by_name('username').send_keys('hashtagcrawler@gmail.com')
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username'))).send_keys('hashtagcrawler@gmail.com') # 계정 ID 입력
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'password'))).send_keys('dkdlel12') # 계정 비밀번호 입력
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button._0mzm-.sqdOP.L3NKy'))).click() # 로그인하기 버튼 클릭
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.glyphsSpriteUser__outline__24__grey_9.u-__7'))).click() # 내 정보 버튼 클릭
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.-nal3 ')))[1].click() # 팔로우 버튼 클릭
        
        scroll = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'isgrP'))) # 스크롤 범위
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll) # 스크롤 맨 밑으로 내리기
        
        time.sleep(1)
        jsScript = """
        function move_up(element) {
            element.scrollTop = element.scrollTop - 2000;
        }

        function move_down(element) {
            console.log('Position before: ' + element.scrollTop);
            element.scrollTop = element.scrollTop + 1000;
            console.log('Position after: ' + element.scrollTop);
        }

        move_up(arguments[0]);
        """
        driver.execute_script(jsScript, scroll) # 스크롤 절반 올리기
        
        # 스크롤 다시 계속 내리기
        try:
            for i in range(10): # 스크롤 10번 내림
                time.sleep(1)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll)
        except:
            print('scroll error!!')
            
        time.sleep(1)
        html = driver.page_source # html 소스코드 가져오기
        dom = BeautifulSoup(html, 'html.parser') # BeautifulSoup 모듈로 순수 html 소스코드 객체구조 변환
        
        accounts_Id_Lists = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.FPmhX.notranslate._0imsa'))) # 팔로우 계정 ID 경로
        account_Id = [j.text for j in accounts_Id_Lists] # 계정 ID 경로 텍스트로 리스트 저장
        
        # 팔로우 계정ID csv 파일로 저장
        with open('instagram_Accounts_Data.csv', 'w', newline = '') as f:
            csv_Writer = csv.writer(f)
            csv_Writer.writerow(account_Id)
            
        driver.close() # 크롬 브라우저 종료
        
    def instagram_Data_Crawling(self, accounts_Lists, output, path = '/Users/markmac/Downloads/chromedriver'):
        self.accounts_Lists = accounts_Lists
        self.output = output
        self.path = path
        
        accounts_Id = [] # 계정ID
        post_Counts = [] # 게시물 수
        follower_Counts = [] # 팔로워 수
        last_Activity_Date = [] # 최근 활동 날짜
        first_Activity_Date = [] # 처음 활동 날짜
        active_Period = [] # 계정 활성 기간
        comment_Counts = [] # 총 댓글 수
        good_Counts = [] # 총 좋아요 수
        avg_Comments = [] # 평균 댓글 수
        avg_Goods = [] # 평균 좋아요 수
        hashtags_Counts = [] # 제일 큰 해시태그/빈도 수
        
        for i in range(len(accounts_Lists)):
            hashtags = [] # 해시태그 html 소스코드 저장
            words = [] # 해시태그와 계정태그 저장
            hashtags_data = [] # 해시태그 저장
            comment = 0
            good = 0
            
            driver = webdriver.Chrome(path)
            driver.get('https://www.instagram.com/' + accounts_Lists[i])
            
            time.sleep(1)
            html = driver.page_source
            dom = BeautifulSoup(html, 'html.parser')
            
            accounts_Id.append(dom.select('h1._7UhW9.fKFbl.yUEEX.KV-D4.fDxYl')[0].text) # 계정ID 저장
            post_Counts.append(int(dom.select('span.g47SY')[0].text.replace(',', ''))) # 게시물 수 저장
            follower_Counts.append(int(dom.select('span.g47SY')[1]['title'].replace(',', ''))) # 팔로워 수 저장
            
            driver.find_element_by_class_name('_9AhH0').click() # 첫번째 게시물 클릭
            
            while True:
                try:
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')
                    
                    # 댓글 더 보기 버튼이 사라지면
                    if dom.select('li.lnrre') == []:
                        # 첫 게시물이 동영상 게시물일때
                        if dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good == 0:
                            time.sleep(1)
                            last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 최근 활동 날짜 수집
                            
                            last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 최근 활동 년도
                            last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 최근 활동 월
                            last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 최근 활동 일
                            
                            comment = len(dom.select('li.gElp9')) # 댓글 저장
                            
                            hashtags += dom.select('div.C4VMK > span > a') # 해시태그 소스코드 저장
                            
                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'vcOH2'))).click() # 조회수 클릭
                            
                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')
                            
                            good = int(dom.select('div.vJRqr > span')[0].text.replace(',', '')) # 좋아요 갯수 저장
                            
                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'QhbhU'))).click() # 게시물 클릭 후 나감
                            
                            # 다음 게시물이 없을 때
                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 처음 활동 날짜 수집

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 첫 활동 년도
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 첫 활동 월
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 첫 활동 일

                                last = date(last_Year, last_Month, last_Day) # 최근활동날짜
                                first = date(first_Year, first_Month, first_Day) # 처음활동날짜

                                period = last - first # 활성기간
                                active_Period.append(period.days) # 활성기간 저장

                                comment_Count.append(comment) # 총 댓글 수 저장
                                good_Count.append(good) # 총 좋아요 수 저장
                                avg_Comment.append(round(comment / post_Count[0])) # 평균 댓글 수 저장
                                avg_Good.append(round(good / post_Count[0])) # 평균 좋아요 수 저장

                                words = [j.text for j in hashtags] # 해시태그 + 계정태그 저장
                                hashtags_data = [x for x in words if x[0] == '#'] # 순수 해시태그 추출
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1]) # 제일 빈도가 큰 해시태그 저장
                                
                                driver.close() # 브라우저 닫기
                                break
                            
                            # 다음 게시물이 있으면
                            else:
                                # 다음 버튼 클릭
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                        
                        # 좋아요가 0인 영상 게시물일때
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and good == 0 and comment != 0:
                            time.sleep(1)
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'vcOH2'))).click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good = int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'QhbhU'))).click()
                            
                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                        # 댓글수가 0일때 영상 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good != 0:
                            time.sleep(1)
                            comment = len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'vcOH2'))).click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'QhbhU'))).click()
                            
                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                        
                        # 댓글, 좋아요 수가 0이 아닐때 영상 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment != 0 and good != 0:
                            time.sleep(1)
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'vcOH2'))).click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'QhbhU'))).click()

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                        # 첫 사진 게시물일때
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good == 0:
                            time.sleep(1)
                            last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                            last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                            last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                            last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                            comment = len(dom.select('li.gElp9'))
                            hashtags += dom.select('div.C4VMK > span > a')
                            good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                        
                        # 댓글 수가 0일때 사진 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good != 0:
                            comment = len(dom.select('li.gElp9'))
                            hashtags += dom.select('div.C4VMK > span > a')
                            good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                        # 좋아요 수가 0일때 사진게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and good == 0 and comment != 0:
                            comment += len(dom.select('li.gElp9'))
                            hashtags += dom.select('div.C4VMK > span > a')
                            good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                        # 둘다 0일때 사진 게시물
                        else:
                            comment += len(dom.select('li.gElp9'))
                            hashtags += dom.select('div.C4VMK > span > a')
                            good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]
                                hashtags_data = [x for x in words if x[0] == '#']
                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                
                    # 댓글 더보기 버튼이 있으면
                    else:
                        time.sleep(1)
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.Z4IfV._0mzm-.sqdOP.yWX7d'))).click() # 댓글 더 보기 버튼 클릭
                except:
                    break
                
        total_data = {
            '계정ID':accounts_Id, '게시물수':post_Counts, '팔로워수':follower_Counts, '평균댓글수':avg_Comments, 
            '평균좋아요수':avg_Goods, '총댓글수':comment_Counts, '총좋아요수':good_Counts, 
            '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '계정활성기간':active_Period, 
            '제일많이나온해시태그':hashtags_Counts
        }

        output.put(total_data)
        
    def multiprocessing_Crawling(self):
        accounts_Lists = []
        accounts_Id_Lists_1 = []
        accounts_Id_Lists_2 = []
        accounts_Id_Lists_3 = []
        accounts_Id_Lists_4 = []
        accounts_Id_Lists_5 = []
        accounts_Id_Lists_6 = []
        instagram_Data = []
        procs = []
        
        # 인스타그램 팔로우 계정ID 데이터 csv파일 읽어오기
        with open('instagram_Accounts_Data.csv', 'r', newline = '') as f:
            csv_Reader = csv.reader(f)
            
            for row in csv_Reader:
                accounts_Lists += row
                
        # multiprocessing 모듈을 사용하기 위해 수집한 계정데이터 분리
        accounts_Id_Lists_1 += accounts_Lists[:9]
        accounts_Id_Lists_2 += accounts_Lists[9:18]
        accounts_Id_Lists_3 += accounts_Lists[18:28]
        accounts_Id_Lists_4 += accounts_Lists[28:38]
        accounts_Id_Lists_5 += accounts_Lists[38:48]
        accounts_Id_Lists_6 += accounts_Lists[48:57]
        
        output1 = Queue()
        output2 = Queue()
        output3 = Queue()
        output4 = Queue()
        output5 = Queue()
        output6 = Queue()

        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_1, output1)))
        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_2, output2)))
        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_3, output3)))
        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_4, output4)))
        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_5, output5)))
        procs.append(Process(target = instagram.instagram_Data_Crawling, args = (accounts_Id_Lists_6, output6)))

        for p in procs:
            p.start()

        instagram_Data += output1.get()
        instagram_Data += output2.get()
        instagram_Data += output3.get()
        instagram_Data += output4.get()
        instagram_Data += output5.get()
        instagram_Data += output6.get()

        for p in procs:
            p.join()
            
        return instagram_Data
            
    def save_Data_Csv(self, instagram_Data):
        self.instagram_Data = instagram_Data
        
        df = pd.DataFrame.from_dict(instagram_Data)
        df.to_csv('/Users/markmac/Downloads/instagram_data.csv')

In [2]:
instagram = instagram_Data_Crawler()

In [3]:
instagram.instagram_Follows_Crawling()

In [4]:
instagram_Dict_Data = instagram.multiprocessing_Crawling()

Process Process-4:
Process Process-3:
Process Process-1:
Process Process-6:
Process Process-5:
Process Process-2:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [5]:
instagram_Dict_Data

NameError: name 'instagram_Dict_Data' is not defined

In [ ]:
instagram.save_Data_Csv(instagram_Dict_Data)